**Rationale** Hypothesis testing is foundational to the entire field of statistics. Without the Central Limit Theorem and hypothesis testing, we would not have modern day (social) science. In marketing, this is generally most important when doing A/B testing of ads. In this assignment, you will practice computing basic summary statistics and conducting hypothesis tests.

[Datasets](https://drive.google.com/drive/folders/1kf6BHlmF32UjsHw6wh_sMJ-JWkV091XD?usp=sharing) required:
1. [FB ad campaign data](https://drive.google.com/file/d/1XqPgu_rFZZ5eHCoFS7drFsmqd7U81vj_/view?usp=sharing).
1. [Starbucks Promos](https://drive.google.com/file/d/1aUJUdGm-RdHaJCJvS1kay9fRdNTq22xb/view?usp=sharing)

In [ ]:
j = 0
k = 0
while k<-1:
    if j%2 == 0:
        j+=1
    else:
        j+=2
    if j>5:
        k = 0
print(j)

0


# 1. (15 points) FB Ad campaigns

1. Use a groupby operation create a dataframe called `sumstats` consisting of the total impressions and clicks for each `xyz_campaign_id`.
1. Using `sumstats`, create a new column called `ctr` that represents the [click through rate](https://support.google.com/google-ads/answer/2615875?hl=en) for each campaign. Which campaign had the highest CTR?
1. Create a column for `sd` for standard deviation of the click through rate.

Hint: Recall from the notes that the standard deviation of a binomial distribution with $Pr(Success) = p$ is $SD(p) = \sqrt{p(1-p)}$. So if the probability of a binary outcome (such as clicking on an ad) is observed to be .3, the standard deviation is $\sqrt{.3(.7)}=\sqrt{.21}$.

In [ ]:
# imports and mount google drive
import os, pandas as pd, numpy as np
from scipy import stats

from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
# set the path to the datasets for A4
fpath = 'drive/MyDrive/Data Analytics/A4/'
os.listdir(fpath)

['starbucks_promos.csv', 'facebook_ads.csv', 'Copy of A4 Stats Review.ipynb']

In [ ]:
# read in the facebook_ads.csv file as the dataframe variable named fb
fb = pd.read_csv(fpath+'facebook_ads.csv')

In [ ]:
# Take a look at the first 5 rows
# Observe that each row represents 1 ad from 1 campaign and a target demographic group (age, gender, interest)
fb.head()

,ad_id,xyz_campaign_id,fb_campaign_id,age,gender,interest,Impressions,Clicks,Spent,Total_Conversion,Approved_Conversion
0,708746,916,103916,30-34,M,15,7350,1,1.43,2,1
1,708749,916,103917,30-34,M,16,17861,2,1.82,2,0
2,708771,916,103920,30-34,M,20,693,0,0.00,1,0
3,708815,916,103928,30-34,M,28,4259,1,1.25,1,0
4,708818,916,103928,30-34,M,28,4133,1,1.29,1,1


In [ ]:
# use a groupby to create new dataframe, sumstats that tabulates the total clicks and impressions for each campaign.
# Generic groupby syntax: sumstats = df.groupby('groupbyvariable')[['summary_variable1', 'summary_variable2']].summaryfunction().reset_index()
# use the correct dataframe name, variables (i.e. column names) and summaryfunction
# Hint: Analyze the different campaigns in 'xyz_campaign_id'
sumstats = fb.groupby('xyz_campaign_id')[['Clicks','Impressions']].sum().reset_index()

In [ ]:
# create a new column in sumstats called 'ctr' (short for click through rate)
# recall that ctr = clicks / impressions
sumstats['ctr'] = sumstats.Clicks / sumstats.Impressions


In [ ]:
# take a look at the whole sumstats dataframe
sumstats


,xyz_campaign_id,Clicks,Impressions,ctr
0,916,113,482925,0.000234
1,936,1984,8128187,0.000244
2,1178,36068,204823716,0.000176


In [ ]:
# create a new column ctr_sd for the std. dev of the ctr (See hint above)
# pay attention to PEMDAS, esp. that you want to take the sqrt after computing p*(1-p)
# np.sqrt(x) returns the sqrt of some number x

sumstats['ctr_std'] = np.sqrt((sumstats.ctr*(1-sumstats.ctr)))


In [ ]:
# take another look at the whole sumstats dataframe
# compare the ctr_std to calculating it by hand, make sure you did this correctly.
sumstats


,xyz_campaign_id,Clicks,Impressions,ctr,ctr_std
0,916,113,482925,0.000234,0.015295
1,936,1984,8128187,0.000244,0.015621
2,1178,36068,204823716,0.000176,0.013269


## Answer
The campaign with the highest CTR is: **936**


# 2. (15 points) Hypothesis testing and confidence intervals

1. Compute the 95% confidence interval for the CTR for each campaign. Compare the confidence intervals of campaign 916 and 936. What can you conclude about the relative perfrmance of the 2 ads in the population (e.g. are they very different, similar, etc.)? How about 916 vs. 1178?
1. Was campaign 936 statistically different compared to campaign 916? How about 936 vs. 1178? Use the `ttest_2sample` function from the notes. Remember, you must define it in your Colab session in order to use it (execute the cell w/ the function).

Given these statistical tests what would you recommend in terms of allocation of the ad budget?

In [ ]:
# use stats.norm.ppf([.025, .975], mean, std error)
# where mean is sample mean and std error is std dev / sqrt(obs) (from Central Limit Theorem)
sumstats['std_error'] = sumstats.ctr_std / np.sqrt(sumstats.Impressions)
# confidence interval for ad 916
def hum(h):
  return (stats.norm.ppf([.025, .975], (sumstats['ctr'][h]), (sumstats['std_error'][h])))
CI_916 = hum(0)
CI_916

array([0.00019085, 0.00027713])

In [ ]:
# confidence interval for ad 936
CI_936 = hum(1)
CI_936

array([0.00023335, 0.00025483])

In [ ]:
# confidence interval for ad 1178
CI_1178 = hum(2)
CI_1178

array([0.00017428, 0.00017791])

**Edit this cell**

1. The confidence interval range of the 916 data set is larger than the 936 data set. Yet, 916 reaches higher peaks and lower lows.
1. The confidence interval range of the 916 is much larger than the 1178 data set.

In [ ]:
# Use this custom funciton

def ttest_2sample(m1,sd1,N1,m2,sd2,N2, twotail = True, equalvar = False):

    """
    This function requires you to supply:
    m1: mean of sample 1
    sd1: std. dev of sample 1
    N1: number of obs of sample 1
    m2: mean of sample 2
    sd2: std dev of sample 2
    N2: number of obs of sample 2

    Optional inputs:
    twotail = True (default) / False. If False, then 1 tail
    equalvar = True / False (default). If True, assumes equal population variance.
    """

    # The difference between equal and unequal variance is only in how to compute
    # the test statistic and degree of freedom.
    if equalvar:
        spsquare = ((N1-1)*sd1**2+(N2-1)*sd2**2)/(N1+N2-2)
        T = (m1-m2)/np.sqrt(spsquare*(1/N1+1/N2))
        nu = N1+N2-2
    else:
        nu = (sd1**2/N1+sd2**2/N2)**2/((sd1**2/N1)**2/(N1-1)+(sd2**2/N2)**2/(N2-1)) # new degree of freedom
        T = (m1-m2)/(np.sqrt(sd1**2/N1+sd2**2/N2))

    # If the first mean is bigger, we need to do 1- cdf
    # Otherwise just compute cdf
    if m1>m2:
        pval = 1-stats.t.cdf(T, df = nu)
    else:
        pval = stats.t.cdf(T, df = nu)

    # return p values
    # If 2 tail, we must multiply by 2
    # otherwise we just return the computed pval
    if twotail == True:
        return pval*2
    else:
        return pval

In [ ]:
# for each campaign grab the Pr(Success) = p which is ctr, std dev, and observations.
"""
    This function requires you to supply:
    m1: mean of sample 1 or ctr
    sd1: std. dev of sample 1
    N1: number of obs of sample 1
    m2: mean of sample 2
    sd2: std dev of sample 2
    N2: number of obs of sample 2
"""
#916
p916 = float(sumstats.loc[sumstats.xyz_campaign_id==916, 'ctr'])
sd916 = np.sqrt(p916*(1-p916))
obs916 = float(sumstats.loc[sumstats.xyz_campaign_id==916, 'Impressions'])
#936
p936 = float(sumstats.loc[sumstats.xyz_campaign_id==936, 'ctr'])
sd936 = np.sqrt(p936*(1-p936))
obs936 = float(sumstats.loc[sumstats.xyz_campaign_id==936, 'Impressions'])
#1178
p1178 = float(sumstats.loc[sumstats.xyz_campaign_id==1178, 'ctr'])
sd1178 = np.sqrt(p1178*(1-p1178))
obs1178 = float(sumstats.loc[sumstats.xyz_campaign_id==1178, 'Impressions'])
# run a 2 tail 2 sample t-test to compare the population CTRs for 936 and 916
ttest_2sample(p916,sd916,obs916,p936,sd936,obs936, twotail = True, equalvar = False)


0.6561623087350006

In [ ]:
# run a 2 tail 2 sample t-test to compare the population CTRs for 936 and 1178
ttest_2sample(p1178,sd1178,obs1178,p936,sd936,obs936, twotail = True, equalvar = False)

2.0046821640920726e-34



Resources should be allocated between all three since 936 and 1178 data set's relationships aren't significant while 916 and 936 are statstically significant. This will allow the best range of advertising.

# 3. (20 points)

Using the Starbucks dataset, conduct an appropriate statistical test to determine:
1. Is there any statistically appreciable difference in the redemption rates of BOGO vs. Discount promotions?
1. Among those who redeemed an offer and reported their income, is there any statistical difference in the average income associated with BOGO vs. discount redemptions? Note that not every observation reports income (select only the observations that do).

In [ ]:
# read in the starbucks_promos.csv file as the dataframe variable named sb, use index_col = 0 option
sb = pd.read_csv(fpath+"starbucks_promos.csv",index_col = 0)

In [ ]:
# preview first 5 rows to get a sense of the contents of the dataframe
# note that each observation represents an offer received by a customer
sb.head()

,uid,event,time,gender,age,register_date,income,offer_id,offer_reward,channels,difficulty,duration,offer_type,offer_time,transaction_amount,redeem_time,redeemed
1,0020c2b971eb4e9188eac86d93036a77,offer received,0,F,59,20160304,90000.0,fafdcd668e3743c1bb461111dcafc2a4,2.0,"['web', 'email', 'mobile', 'social']",10.0,240.0,discount,0.0,17.63,54.0,1
4,005500a7188546ff8a767329a2f7c76a,offer received,0,M,56,20171209,47000.0,ae264e3637204a6fb9bb56bc8210ddfd,10.0,"['email', 'mobile', 'social']",10.0,168.0,bogo,0.0,NaN,NaN,0
5,0056df74b63b4298809f0b375a304cf4,offer received,0,M,54,20160821,91000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,"['web', 'email', 'mobile']",5.0,168.0,bogo,0.0,27.86,132.0,1
6,00715b6e55c3431cb56ff7307eb19675,offer received,0,F,58,20171207,119000.0,ae264e3637204a6fb9bb56bc8210ddfd,10.0,"['email', 'mobile', 'social']",10.0,168.0,bogo,0.0,27.26,12.0,1
8,00840a2ca5d2408e982d56544dc14ffd,offer received,0,M,26,20141221,61000.0,2906b810c7d4411798c6938adc9daaa5,2.0,"['web', 'email', 'mobile']",10.0,168.0,discount,0.0,6.05,540.0,1


In [ ]:
# select the redeemed column for discount offers types, name this data the variable disc
# ( e.g. disc = sb.loc[sb.offer_type=='discount', 'redeemed'] )
disc = sb.loc[sb.offer_type=='discount', 'redeemed']


In [ ]:
# select the redeemed column for buy one get one (bogo) offers, name this data the variable bogo
bogo = sb.loc[sb.offer_type=='bogo', 'redeemed']


In [ ]:
# use stats.ttest_ind to test whether the 2 types of offers yield different redemption rates
ttest = stats.ttest_ind(bogo,disc)
ttest

TtestResult(statistic=-17.30160421286854, pvalue=6.611390792545181e-67, df=61040.0)

In [ ]:
# Select the income column for those who redeemed the BOGO offer, call this the variable bogo_income
# note you need to use 3 condtitions for the row selection: one for redeemed, one for bogo offer, and one for income not missing
# FYI: dataframe.somecolumn.notnull() is the Boolean evaluation for whether the somecolumn is not missing data
bogo_income = sb.loc[(sb.offer_type=='bogo')&(sb.redeemed==1)&(sb.income.notnull()==True), 'income']


In [ ]:
# Select the income column for those who redeemed the discount offer, call this the variable disc_income
# note you need to use 3 condtitions for the row selection: one for redeemed, one for discount offer,  and one for income not missing
# FYI: dataframe.somecolumn.notnull() is the Boolean evaluation for whether the somecolumn is not missing data
disc_income = sb.loc[(sb.offer_type=='discount')&(sb.redeemed==1)&(sb.income.notnull()==True), 'income']


In [ ]:
# What is the average income for bogo redeemers (bogo_income.mean())?
bogo_mean = bogo_income.mean()
bogo_mean

70114.8559593297

In [ ]:
# What is the average income for discount redeemers?
disc_mean = disc_income.mean()
disc_mean

68617.39130434782

In [ ]:
# use stats.ttest_ind to test whether the average income of the redeemers of the 2 offer types are different
ttest2 = stats.ttest_ind(bogo_income,disc_income)
ttest2

TtestResult(statistic=6.343377852366808, pvalue=2.2763463492984577e-10, df=33641.0)


1. The relationship between the different offers being redeemed is statistically significant since the t-test was -17 with a p-value of essentially 0.
1. The relationship between the different offers based off of income is statistically significant since the t-test was 6.34 with a p-value of essentially 0.